In [1]:
import os 
import os.path as osp
import glob
import re
import numpy as np 
from shutil import copyfile
from datetime import datetime

In [2]:
"""
List set of patients
Store information of patients in a dict (id, days, imgs)
"""    
PATH ="dataset/data_origin/Individual patients/"
"""
Filter images from patient_set"""
def patient_set(path):
    patients = []
    dict_patients = {}
    list_img = os.listdir(path)
    for i in list_img:
        if "patient" in i.lower(): 
            patients.append(i)
    for p in patients:
        #p_id in -1 position
        id = p.split(" ")[-1]
        dict_patients[id] = dict()
        p_path = osp.join(path, p)
        for i in os.listdir(p_path):
            if os.path.isdir(os.path.join(p_path, i)):
                p_path = os.path.join(p_path,i)
                break
        days = []
        imgs = []
        for img in os.listdir(p_path):
            if "ods" in img or "measure" in img or "laser" in img or "annotated" in img or "scaled" in img or "depth" in img:
                continue
            imgs.append(osp.join(p_path,img))
            match = re.search(r'\d{4}-\d{2}-\d{2}', img)
            days.append(datetime.strptime(match.group(), '%Y-%m-%d').date())
        dict_patients[id]["days"] = list(np.unique(days))
        dict_patients[id]["imgs"] = imgs
    return patients, dict_patients

In [3]:
p_set, p_dict = patient_set(PATH)


In [17]:
def copy_image(p_dict,path_new):
    for p in p_dict:
        p_path = os.path.join(path_new, p)
        if not os.path.exists(p_path):
            os.makedirs(p_path,exist_ok = True)
        count = dict()
        for d in p_dict[p]["days"]:
            count[str(d)] = 0
        for img in p_dict[p]["imgs"]:
            day = re.search(r'\d{4}-\d{2}-\d{2}', img)
            day = datetime.strptime(day.group(), '%Y-%m-%d').date()
            if "jpeg" in img:
                count[str(day)] = 100
                copyfile(img, os.path.join(p_path, str(day) + ".jpg"))
            if (str(day)+".jpg" in os.listdir(p_path)) and (not count[str(day)] == 100):
                count[str(day)] +=1
                copyfile(img, os.path.join(p_path, str(day) + "_index_{}.jpg".format(count[str(day)])))
#             else:
#                 count[str(day)] +=1
#                 copyfile(img, os.path.join(p_path, str(day) + "_index_{}.jpg".format(count[str(day)])))
        

In [18]:
copy_image(p_dict, "img_filter_4")